In [1]:
"""
Error Code:
    0: Error opening file
"""

import pandas as pd
import numpy as np
import zipfile

# Clarify the mission N number for this .ipynb
N_number = 1

In [2]:
def namestr(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]

def display(items):
    print(namestr(items, globals()))
    for item in items:
        print("    " + item)

# Test display
test_dict = {"A": [1, 2, 3], "B": [4, 5, 6]}
display(test_dict)

['test_dict']
    B
    A


In [3]:
from os import listdir
from os.path import isfile, join

mypath = "./"
zip_file_name = "receipt-id-773931-part-001"
zip_file = zipfile.ZipFile(mypath + zip_file_name + ".zip")
file_name_list = zip_file.namelist()
# Should return 4 folders if testing on ASG Zip
print("{0} files detected under current directory. ".format(len(file_name_list)))

53528 files detected under current directory. 


In [4]:
def filter_by_filename(files_list):
    """Function filtering files by filenames.
    Only accepts files starts with "journal-article"
    """
    filtered_list = []
    mask_match = "ngram" + str(N_number) + "/journal-article"
    for filename in files_list:
        # Check if the filename starts with "journal-article"
        assert isinstance(filename, str)
        # Check the first 20 characters of the file name
        if filename.startswith(mask_match, 0, len(mask_match)):
            filtered_list.append(filename)
    return filtered_list

# Run "filter_by_filename" for current directory
filtered_list = filter_by_filename(file_name_list)
print("{0} files remained after filename filtering. ".format(len(filtered_list)))

13382 files remained after filename filtering. 


In [5]:
# Functions checking word attributes (single-letter, starts/ends with numebr)
def is_single_letter(word):
    assert isinstance(word, str)
    return len(word) <= 1

def starts_with_number(word):
    assert isinstance(word, str)
    try:
        return word[0].isdigit()
    except:
        return False

def ends_with_number(word):
    assert isinstance(word, str)
    try:
        return word[len(word) - 1].isdigit()
    except:
        return False

# Summary of check functions
check_funcs = [
    is_single_letter, 
    starts_with_number, 
    ends_with_number,
]

In [6]:
file_0 = filtered_list[0]
try:
    journal_0 = zip_file.open(file_0, mode="r")
except IOError:
    print("Error opening file {0}".format(file_0))
    exit(0)

In [7]:
# Track the meaningful dictonaries
freq_dict = {}

# Read file line by line
for line in journal_0:
    line = line.decode("ascii") 
    assert isinstance(line, str)
    pair = line.strip().split()
    
    # Separate word/freq
    word, freq = pair
    assert isinstance(word, str)
    
    # Filter by word's attribute
    check_results = [check_func(word) for check_func in check_funcs]
    if any(check_results):
        continue
    freq_dict.update({word: freq})

journal_0.close()
display(freq_dict)

['freq_dict']
    given
    found
    solving
    described
    complexity
    processing
    influ
    maier
    levine
    brooks
    stimulate
    behavior
    accepted
    letters
    calls
    types
    centers
    depending
    nonperson
    comparative
    hospital
    cular
    stipulate
    re
    sets
    specify
    non
    ue
    fail
    domain
    departures
    half
    preferred
    distribution
    relate
    smaller
    measures
    es
    unstructured
    articulate
    panels
    qualifications
    understood
    dorsey
    cedures
    lacking
    analytic
    audits
    informational
    would
    credibility
    tin
    informal
    we
    four
    chologists
    nature
    consultant
    base
    mathematical
    measure
    formance
    amort
    vary
    type
    national
    cause
    july
    personnel
    positive
    erogeneous
    those
    enhancing
    aggregate
    achieved
    utes
    example
    trol
    project
    implement
    pose
    trates
    

In [8]:
# Create Dataframe
# Data
data = []
for word in freq_dict:
    freq = freq_dict[word]
    data.append([word, freq])

# Columns
columns = ["word", "freq"]

# Index
index = list(range(len(freq_dict)))

# DataFrame
dataframe = pd.DataFrame(data, columns=columns, index=index)

# Print Test
dataframe.head()

,word,freq
0,given,3
1,found,7
2,solving,2
3,described,9
4,complexity,8


In [11]:
# Run filtering for each file
progress = list(range(0, len(filtered_list), len(filtered_list) // 99))

for file_num in range(0, len(filtered_list)):
    
    file = filtered_list[file_num]
    # Open file
    try:
        file_open = zip_file.open(file, mode="r")
    except IOError:
        print("Error opening file {0}".format(file))
        exit(0)
    
    # Create output file
    output_name = mypath + "Results_1/" + file[len("/ngram" + str(N_number)):]
    
    # Note: Output file encoding set to UTF-8
    output_file = open(output_name, mode="w+", encoding="utf-8")
    
    # Track for max length word
    # Used for formatting output data
    max_length = float("-inf")
        
    # Initiate freq_dict -> {word: freq}
    freq_dict = {}
    
    # Read by line
    for line in file_open:
        # Line decode using UTF-8
        line = line.decode("utf-8")
        assert isinstance(line, str)
        pair = line.strip().split()
    
        # Separate word/freq
        word, freq = pair
        assert isinstance(word, str)
        
        # Filter by word's attribute
        check_results = [check_func(word) for check_func in check_funcs]
        if any(check_results):
            continue
            
        # Update max_length
        if len(word) > max_length:
            max_length = len(word)
        freq_dict.update({word: freq})
    
    # Close reading file
    file_open.close()
    
      """ [START] Create Dataframe """ 
        
#     # Data
#     data = []
#     for word in freq_dict:
#         freq = freq_dict[word]
#         data.append([word, freq])
    
#     # Columns
#     columns = ["word", "freq"]
    
#     # Index
#     index = list(range(len(freq_dict)))
    
#     # DataFrame
#     dataframe = pd.DataFrame(data, columns=columns, index=index)

    """ [END] Create Dataframe """ 
    
    # Write to output file
    for word in freq_dict:
        freq = freq_dict[word]
        output_file.write(word.ljust(max_length + 5))
        output_file.write(freq + "\n")
    
    output_file.close()
    
    # Track Progress
    if len(progress) > 0 and file_num >= progress[0]:
        print("Progress: {}%".format(100 - len(progress)))
        progress.pop(0)

# Print Sample Output
print(dataframe.head()) 

Progress: 0%
Progress: 1%
Progress: 2%
Progress: 3%
Progress: 4%
Progress: 5%
Progress: 6%
Progress: 7%
Progress: 8%
Progress: 9%
Progress: 10%
Progress: 11%
Progress: 12%
Progress: 13%
Progress: 14%
Progress: 15%
Progress: 16%
Progress: 17%
Progress: 18%
Progress: 19%
Progress: 20%
Progress: 21%
Progress: 22%
Progress: 23%
Progress: 24%
Progress: 25%
Progress: 26%
Progress: 27%
Progress: 28%
Progress: 29%
Progress: 30%
Progress: 31%
Progress: 32%
Progress: 33%
Progress: 34%
Progress: 35%
Progress: 36%
Progress: 37%
Progress: 38%
Progress: 39%
Progress: 40%
Progress: 41%
Progress: 42%
Progress: 43%
Progress: 44%
Progress: 45%
Progress: 46%
Progress: 47%
Progress: 48%
Progress: 49%
Progress: 50%
Progress: 51%
Progress: 52%
Progress: 53%
Progress: 54%
Progress: 55%
Progress: 56%
Progress: 57%
Progress: 58%
Progress: 59%
Progress: 60%
Progress: 61%
Progress: 62%
Progress: 63%
Progress: 64%
Progress: 65%
Progress: 66%
Progress: 67%
Progress: 68%
Progress: 69%
Progress: 70%
Progress: 71%
Pr